In [ ]:
import os
import numpy as np
import rawpy
import imageio
from PIL import Image
import imgaug.augmenters as iaa

# Define augmentation pipeline
seq = iaa.Sequential([
    iaa.Fliplr(0.5),   # horizontally flip 50% of the images
    iaa.Flipud(0.5),   # vertically flip 50% of the images
    iaa.Affine(rotate=(-10, 10)),  # rotate by -10 to +10 degrees
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # add gaussian noise
    iaa.GaussianBlur(sigma=(0, 3.0)),  # blur images with a sigma of 0 to 3.0
    iaa.Grayscale(alpha=(0.0, 1.0)),  # convert images to grayscale
    iaa.Sometimes(0.5, iaa.ChannelShuffle(1.0)),  # shuffle channels
    iaa.Sometimes(0.5, iaa.Invert(1.0)),  # invert pixel values
    iaa.Sometimes(0.5, iaa.ContrastNormalization((0.5, 2.0))),  # contrast normalization
    iaa.Sometimes(0.5, iaa.Multiply((0.5, 1.5), per_channel=0.5)),  # multiply pixel values
    iaa.Sometimes(0.5, iaa.LinearContrast((0.5, 2.0))),  # linear contrast adjustment
    iaa.Sometimes(0.5, iaa.HistogramEqualization()),  # histogram equalization
    iaa.Sometimes(0.5, iaa.GammaContrast((0.5, 2.0))),  # gamma contrast adjustment
    iaa.Sometimes(0.5, iaa.CLAHE()),  # contrast limited adaptive histogram equalization
    iaa.Sometimes(0.5, iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV")),  # RGB to HSV
    iaa.Sometimes(0.5, iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="Lab")), # RGB to LAB
    iaa.Sometimes(0.5, iaa.Grayscale(alpha=(0.0, 1.0))),  # convert images to grayscale
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 1.0))),  # gaussian blur
    iaa.Sometimes(0.5, iaa.MotionBlur(k=15, angle=[-45, 45])),  # motion blur
    iaa.Sometimes(0.5, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 2.0))),  # sharpen images
    iaa.Sometimes(0.5, iaa.AddToBrightness((-50, 50))),  # change brightness of images
    iaa.Sometimes(0.5, iaa.AddToHueAndSaturation((-20, 20))),  # change hue and saturation
    iaa.Sometimes(0.5, iaa.MultiplyHueAndSaturation((0.5, 1.5))),  # multiply hue and saturation
    iaa.Sometimes(0.5, iaa.LinearContrast((0.5, 2.0))),  # linear contrast adjustment
    iaa.Sometimes(0.5, iaa.AllChannelsCLAHE()),  # contrast limited adaptive histogram equalization for all channels
    iaa.Sometimes(0.5, iaa.AllChannelsHistogramEqualization()),  # histogram equalization for all channels
    iaa.Sometimes(0.5, iaa.AddToHue((-50, 50))),  # change hue of images
    iaa.Sometimes(0.5, iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6))),  # sigmoid contrast adjustment
    iaa.Sometimes(0.5, iaa.FastSnowyLandscape(lightness_threshold=(128, 255))),  # snowy landscape effect
    iaa.Sometimes(0.5, iaa.Superpixels(p_replace=(0.1, 1.0), n_segments=(16, 128))),  # superpixels
    iaa.Sometimes(0.5, iaa.Cartoon()),  # cartoon effect
    iaa.Sometimes(0.5, iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))),  # emboss effect
    iaa.Sometimes(0.5, iaa.AverageBlur(k=(2, 7))),  # average blur
    iaa.Sometimes(0.5, iaa.MedianBlur(k=(3, 11))),  # median blur
    iaa.Sometimes(0.5, iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)),  # elastic transformation
    iaa.Sometimes(0.5, iaa.PerspectiveTransform(scale=(0.01, 0.1))),  # perspective transformation
    iaa.Sometimes(0.5, iaa.PiecewiseAffine(scale=(0.01, 0.05))),  # piecewise affine transformation
    iaa.Sometimes(0.5, iaa.CoarseDropout(0.02, size_percent=0.1)),  # coarse dropout
    iaa.Dropout(0.02),
    iaa.Sometimes(0.5, iaa.Cutout(nb_iterations=(1, 3), size=0.2, squared=False)),  # cutout
    iaa.AddToBrightness((-50, 50)), # Adjust brightness by -50 to 50
    iaa.LinearContrast((0.5, 2.0)), # Adjust contrast by a factor between 0.5 and 2.0
    iaa.AverageBlur(k=(2, 7)), # Apply average blur with a kernel size between 2 and 7
    iaa.Sometimes(0.5, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 2.0))),  # sharpen
])

# Calculate total number of augmented images
total_images = sum(len(files) for _, _, files in os.walk("dataset"))
target_images = 2000
augmentation_factor = target_images / total_images

# Augment images to reach target number
for root, dirs, files in os.walk("dataset"):
    for image_name in files:
        image_path = os.path.join(root, image_name)
        if image_path.endswith(".cr3"):
            try:
                with rawpy.imread(image_path) as raw:
                    rgb = raw.postprocess()
                image = Image.fromarray(rgb)
            except Exception as e:
                print(f"Error converting {image_path} to jpg: {e}")
                continue
        else:
            try:
                image = Image.open(image_path)
            except Exception as e:
                print(f"Error opening {image_path}: {e}")
                continue
            
        image = np.array(image)

        for i in range(int(augmentation_factor)):
            augmented_images = seq(images=[image])
            for j, augmented_image in enumerate(augmented_images):
                augmented_image = Image.fromarray(augmented_image)
                augmented_image.save(f"augmented_images/{os.path.basename(root)}_{image_name.split('.')[0]}_aug_{i*len(augmented_images) + j}.jpg")
